# 일반 대화 - [모두의 말뭉치: 일상 대화 말뭉치 2022](https://kli.korean.go.kr/corpus/request/corpusRegist.do)

## 데이터 로드

In [77]:
import os
import json
import pandas as pd

class ConversationData:
    def __init__(self, json_data):
        self.json_data = json_data
    
    def is_normal_conversation(self):
        return "일상대화" in self.json_data["metadata"]["category"]
    
    def get_topic_type(self):
        document = self.__get_document()
        full_topic = document["metadata"]["topic"]
        return full_topic.split(" > ")[0]
    
    def get_conversation(self):
        document = self.__get_document()
        
        # 한 발화자가 여러 utterance에 걸쳐 발화를 하는 경우, 하나의 문장으로 만든다.
        conversation = []
        curr_speaker = None
        curr_sentence = None

        for utterance in document["utterance"]:
            sentence = utterance["form"].strip()
            speaker_id = utterance["speaker_id"].strip()
            if sentence == "":
                continue

            # 대화 처음
            if curr_speaker is None:
                curr_speaker = speaker_id
                curr_sentence = sentence
                continue


            # 이전 발화자가 계속 발화를 이어가는 경우
            if curr_speaker == speaker_id:
                curr_sentence += " " + sentence

            # 다른 사람이 발화하는 경우
            else:
                conversation.append(curr_sentence)
                curr_speaker = speaker_id
                curr_sentence = sentence

        conversation.append(curr_sentence)
        
        return conversation
        
    def __get_document(self):
        return self.json_data["document"][0] # document는 size가 1인 list

def load_json(path):
    with open(path) as file:
        return json.load(file)

def load_nikl_normal_conversation_df(data_dir_path="data/NIKL_DIALOGUE_2022_v1.0"):
    topic_types = []
    conversations = []
    
    for file_path in os.listdir(data_dir_path):
        if not file_path.endswith("json"):
            continue
        
        json_data = load_json(f"{data_dir_path}/{file_path}")
        conversation_data = ConversationData(json_data)
        
        if not conversation_data.is_normal_conversation():
            continue
        
        topic_types.append(conversation_data.get_topic_type())
        conversations.append(conversation_data.get_conversation())

    return pd.DataFrame({
        "topic_type": topic_types,
        "conversation": ["\n".join(c) for c in conversations],
    })


In [78]:
nikl_normal_conversation_df = load_nikl_normal_conversation_df()

## 데이터 수 확인

In [79]:
# 전체 데이터 수
nikl_normal_conversation_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1767 entries, 0 to 1766
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   topic_type    1767 non-null   object
 1   conversation  1767 non-null   object
dtypes: object(2)
memory usage: 27.7+ KB


In [80]:
# 주제별 데이터
nikl_normal_conversation_df["topic_type"].value_counts()

대중교통         122
방송/연예        121
취직           120
건강/다이어트      116
경제/재테크       114
먹거리          113
회사/학교        111
휴가           111
스포츠/레저/취미    108
가족/관혼상제      108
쇼핑           106
생활/주거환경      104
우정           104
반려동물         104
음악           103
기타           102
Name: topic_type, dtype: int64

In [81]:
# random data
nikl_normal_conversation_df.sample(20)

,topic_type,conversation
1029,취직,어 요즘은 취직이 그 상당히 인제 그게 인제 사람 수명이 길어지면서 어 직장을 하나...
776,방송/연예,최근에 영화관 가격이 되게 많이 상향이 돼 가지고 사람들이 오티티를 많이 이용하잖아...
1374,가족/관혼상제,요즘 그 가족 간의 모임이나 가족들이 모였을 때나 이랬을 때 인제 서로 호칭을 부르...
58,방송/연예,그 그저껜가 히든싱어 왕중왕전 했었잖아. 와 그 내 걔 누구지 1등 한 애? 걔 하...
1335,생활/주거환경,오빠 화장실 청소 어떻게 하고 있어?\n화장실 청소야 그 솔로 1주일에 1번 정도 ...
605,먹거리,아빠 먼저 아빠의 입장에 대해서 이야기를 해 봐. 배달료가 비싸지 않 비싸지 않다 ...
702,건강/다이어트,오늘 아침도 너무 일어나기 힘들더라. 목이 너무 아파서 이걸 어찌 하면 좋을까?\n...
584,건강/다이어트,내가 병원에 가서 인제 건강 검진을 받고 했는데 의사 선생님도 그렇지만은 그렇게 말...
429,기타,사실 이거 수능이 우리 어렸을 땐 되게 큰 시험 같잖냐 넌 물론 수능 뭐 중요하진 ...
1680,음악,어 요즘 노래 요새 듣는 거 있어?\n요즘에? 요즘에는 그냥 티브이에 나오는 거 있...


## 전처리

In [82]:
processed_df = nikl_normal_conversation_df.copy()

### 10문장 미만 데이터 제거

In [83]:
processed_df["conversation_len"] =\
    processed_df["conversation"].apply(lambda x: len(x.split("\n")))

processed_df = processed_df[processed_df["conversation_len"] >= 10]

processed_df

,topic_type,conversation,conversation_len
0,쇼핑,아빠가 주로 사고 싶은 건 별로 없는데 자전거 용품 응 산 용품은 다 많잖아?\n어...,293
1,생활/주거환경,이사를 가야 되는데 이거 우리가 동네를 어디로 옮기면 좋겠나? 지금 거기 거기 있는...,190
2,음악,요즘 자주 듣는 음악은 뭐야?\n나는 이제 검정치마의 노래를 듣고 있어.\n검정치마...,189
3,기타,운전면허증은 그럼 언제 따셨어요?\n저는 21살 땐가 22살 20살인가 21살 때 ...,105
4,건강/다이어트,여보 운동할 때에 당신\n응 응\n모래 주머니 차고 했잖아요.\n그럼\n어때요?\n...,78
...,...,...,...
1762,가족/관혼상제,오빠네 친구들은 뭐 요즘 제사나 이런 거 지내는 거 같아?\n일단 아는 형이 제사를...,216
1763,취직,아 지금도 우리가 친구들을 만나면 옛날 뭐 초창기에 나 어디 회사에 다녔어. 나 어...,45
1764,대중교통,어. 오늘 엄마 어 안 왔대?\n아니 너하고 약속해 가지고 세 우리가 3시까지 만나...,64
1765,취직,내가 옛날에 회사 다녔을 때 그때 약간 사수분이 일을 잘 안 가르쳐 줘 가지고 나 ...,37


### 10문장만 데이터 뽑기

In [84]:
def remove_out_of_n(conversation, n):
    return "\n".join(conversation.split("\n")[:n])

processed_df["conversation"] = processed_df["conversation"].apply(lambda x: remove_out_of_n(x, n=10))
processed_df["conversation"]

0       아빠가 주로 사고 싶은 건 별로 없는데 자전거 용품 응 산 용품은 다 많잖아?\n어...
1       이사를 가야 되는데 이거 우리가 동네를 어디로 옮기면 좋겠나? 지금 거기 거기 있는...
2       요즘 자주 듣는 음악은 뭐야?\n나는 이제 검정치마의 노래를 듣고 있어.\n검정치마...
3       운전면허증은 그럼 언제 따셨어요?\n저는 21살 땐가 22살 20살인가 21살 때 ...
4       여보 운동할 때에 당신\n응 응\n모래 주머니 차고 했잖아요.\n그럼\n어때요?\n...
                              ...                        
1762    오빠네 친구들은 뭐 요즘 제사나 이런 거 지내는 거 같아?\n일단 아는 형이 제사를...
1763    아 지금도 우리가 친구들을 만나면 옛날 뭐 초창기에 나 어디 회사에 다녔어. 나 어...
1764    어. 오늘 엄마 어 안 왔대?\n아니 너하고 약속해 가지고 세 우리가 3시까지 만나...
1765    내가 옛날에 회사 다녔을 때 그때 약간 사수분이 일을 잘 안 가르쳐 줘 가지고 나 ...
1766    너 오늘 서면 올 때 뭐 타고 왔어?\n나는 원래는 버스 타고 오고 싶었는데 늦어서...
Name: conversation, Length: 1762, dtype: object

### 주제 별로 1,000개 데이터 뽑기

In [86]:
new_df = pd.DataFrame({
    "topic_type": [],
    "conversation": [],
    "conversation_len": [],
})

for topic_type, data_num in [
    ("대중교통", 63),
    ("방송/연예", 63),
    ("취직", 63),
    ("건강/다이어트", 63),
    ("경제/재테크", 63),
    ("먹거리", 63),
    ("회사/학교", 63),
    ("휴가", 63),
    ("스포츠/레저/취미", 62),
    ("가족/관혼상제", 62),
    ("쇼핑", 62),
    ("생활/주거환경", 62),
    ("우정", 62),
    ("반려동물", 62),
    ("음악", 62),
    ("기타", 62),
]:
    topic_df = processed_df[processed_df["topic_type"] == topic_type]
    topic_df = topic_df.iloc[:data_num, :]
    new_df = pd.concat([new_df, topic_df], axis=0, ignore_index=True)
    
new_df

,topic_type,conversation,conversation_len
0,대중교통,대중교통은 지하철이 제일 편한 거 같은데 뭔가 대구랑 서울이랑 많이 다르지 않아?\...,59.0
1,대중교통,오늘 전철을 타고 우리가 한 몇 분 정도 걸렸지 여기까지?\n어 전철 탄 시간은 3...,89.0
2,대중교통,요번 수요일 날 어 name1 이네랑 우리 망년회를 했었잖아. 그때 name2 씨가...,69.0
3,대중교통,안녕하세요 음 오늘은 아 대중교통에 대한 그 수도권과 아 지방권에 대해서 차이점에 ...,40.0
4,대중교통,어 어 나 대중교통 탈 때 환승 같은 거 되게 편리하다고 보거든 어 같은 돈 2번 ...,78.0
...,...,...,...
995,기타,여기까지 오는 데 몇 분 걸렸게?\n한 15분 걸렸나?\n운전을 하다 보면 운전은 ...,237.0
996,기타,우리가 요즘 엠비티아이가 대세라서 뭐 요즘 또 인기는 인기는 식었지만 내가 1번 해...,166.0
997,기타,요즘 친구들이랑 재밌는 일 있었던 거 있어?\n나는 이제 수능이 끝나고 애들을 많이...,128.0
998,기타,누나 엠비티아이 검사해 봤어?\n어 나 해 봤어 나 해 봤더니 인터넷에서 검사지로 ...,63.0


## 데이터 저장

In [87]:
data_df = new_df[["topic_type", "conversation"]]
data_df.to_csv("data/normal_conversation_nikl.csv", index=False)

### train 데이터와 합치기

In [88]:
train_df = pd.read_csv("data/train.csv")
train_df

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...,...,...
3945,3945,기타 괴롭힘 대화,준하야 넌 대가리가 왜이렇게 크냐?\n내 머리가 뭐.\n밥먹으면 대가리만 크냐 너는...
3946,3946,갈취 대화,내가 지금 너 아들 김길준 데리고 있어. 살리고 싶으면 계좌에 1억만 보내\n예.?...
3947,3947,직장 내 괴롭힘 대화,나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지.\n왜 그래. 들리겠어...
3948,3948,갈취 대화,누구맘대로 여기서 장사하래?\n이게 무슨일입니까?\n남의 구역에서 장사하려면 자릿세...


In [89]:
normal_conversation_nikl_df = pd.read_csv("data/normal_conversation_nikl.csv")
normal_conversation_nikl_df

,topic_type,conversation
0,대중교통,대중교통은 지하철이 제일 편한 거 같은데 뭔가 대구랑 서울이랑 많이 다르지 않아?\...
1,대중교통,오늘 전철을 타고 우리가 한 몇 분 정도 걸렸지 여기까지?\n어 전철 탄 시간은 3...
2,대중교통,요번 수요일 날 어 name1 이네랑 우리 망년회를 했었잖아. 그때 name2 씨가...
3,대중교통,안녕하세요 음 오늘은 아 대중교통에 대한 그 수도권과 아 지방권에 대해서 차이점에 ...
4,대중교통,어 어 나 대중교통 탈 때 환승 같은 거 되게 편리하다고 보거든 어 같은 돈 2번 ...
...,...,...
995,기타,여기까지 오는 데 몇 분 걸렸게?\n한 15분 걸렸나?\n운전을 하다 보면 운전은 ...
996,기타,우리가 요즘 엠비티아이가 대세라서 뭐 요즘 또 인기는 인기는 식었지만 내가 1번 해...
997,기타,요즘 친구들이랑 재밌는 일 있었던 거 있어?\n나는 이제 수능이 끝나고 애들을 많이...
998,기타,누나 엠비티아이 검사해 봤어?\n어 나 해 봤어 나 해 봤더니 인터넷에서 검사지로 ...


In [90]:
curr_idx = train_df["idx"].max() + 1

for idx, data in normal_conversation_nikl_df.iterrows():
    train_df = train_df.append({
        "idx": curr_idx,
        "class": "일반 대화",
        "conversation": data["conversation"],
    }, ignore_index=True)

    curr_idx += 1
    
train_df

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...,...,...
4945,4945,일반 대화,여기까지 오는 데 몇 분 걸렸게?\n한 15분 걸렸나?\n운전을 하다 보면 운전은 ...
4946,4946,일반 대화,우리가 요즘 엠비티아이가 대세라서 뭐 요즘 또 인기는 인기는 식었지만 내가 1번 해...
4947,4947,일반 대화,요즘 친구들이랑 재밌는 일 있었던 거 있어?\n나는 이제 수능이 끝나고 애들을 많이...
4948,4948,일반 대화,누나 엠비티아이 검사해 봤어?\n어 나 해 봤어 나 해 봤더니 인터넷에서 검사지로 ...


In [91]:
train_df.to_csv("data/train_with_normal_nikl.csv", index=False)

In [92]:
train_with_normal_nikl_df = pd.read_csv("data/train_with_normal_nikl.csv")
train_with_normal_nikl_df

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...,...,...
4945,4945,일반 대화,여기까지 오는 데 몇 분 걸렸게?\n한 15분 걸렸나?\n운전을 하다 보면 운전은 ...
4946,4946,일반 대화,우리가 요즘 엠비티아이가 대세라서 뭐 요즘 또 인기는 인기는 식었지만 내가 1번 해...
4947,4947,일반 대화,요즘 친구들이랑 재밌는 일 있었던 거 있어?\n나는 이제 수능이 끝나고 애들을 많이...
4948,4948,일반 대화,누나 엠비티아이 검사해 봤어?\n어 나 해 봤어 나 해 봤더니 인터넷에서 검사지로 ...


In [93]:
train_with_normal_nikl_df["class"].value_counts()

기타 괴롭힘 대화      1094
일반 대화          1000
갈취 대화           981
직장 내 괴롭힘 대화     979
협박 대화           896
Name: class, dtype: int64